# VacationPy

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bonthe,7.5264,-12.5050,25.16,88,95,2.85,SL,1694987851
1,1,port-aux-francais,-49.3500,70.2167,5.26,96,100,12.94,TF,1694987851
2,2,port alfred,-33.5906,26.8910,12.53,80,0,3.52,ZA,1694987565
3,3,port mathurin,-19.6833,63.4167,22.90,76,2,9.30,MU,1694987853
4,4,invercargill,-46.4000,168.3500,9.45,83,88,4.92,NZ,1694987853


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500
    
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df[
    (city_data_df["Max Temp"]< 27 ) & (city_data_df["Max Temp"] > 21)  
    & (city_data_df["Wind Speed"] < 4.5) 
    & (city_data_df["Cloudiness"] ==0)
    
    
]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,bethel,41.3712,-73.4140,23.44,69,0,1.54,US,1694987863
90,90,corfu,39.6200,19.9197,25.45,83,0,0.00,GR,1694987899
101,101,tamanrasset,22.7850,5.5228,24.95,15,0,1.03,DZ,1694987906
129,129,colonia,50.9333,6.9500,22.76,77,0,2.57,DE,1694987674
140,140,farah,32.5000,62.5000,22.27,19,0,1.74,AF,1694987927


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country","Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
22,bethel,US,41.3712,-73.4140,
90,corfu,GR,39.6200,19.9197,
101,tamanrasset,DZ,22.7850,5.5228,
129,colonia,DE,50.9333,6.9500,
140,farah,AF,32.5000,62.5000,
215,brookings,US,44.3114,-96.7984,
232,revelstoke,CA,50.9832,-118.2023,
238,saint-pierre,RE,-21.3393,55.4781,
253,sedalia,US,38.7045,-93.2283,
285,santa terezinha,BR,-29.9002,-50.0904,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit": 20 
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
       
    

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    #print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
corfu - nearest hotel: Hotel Bella Venezia
tamanrasset - nearest hotel: فندق الأمان
colonia - nearest hotel: Wasserturm Hotel Cologne
farah - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
revelstoke - nearest hotel: Grizz Hotel
saint-pierre - nearest hotel: Tropic Hotel
sedalia - nearest hotel: Hotel Bothwell
santa terezinha - nearest hotel: Pousada Santa Terezinha
al jawf - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
mitchell - nearest hotel: Quality Inn
pamandzi - nearest hotel: Gite Le Madina
toliara - nearest hotel: Ambary
mersa matruh - nearest hotel: صقور
bir el hafey - nearest hotel: No hotel found
cabinda - nearest hotel: Hotel Mayombe
maintirano - nearest hotel: Hotel Safari


,City,Country,Lat,Lng,Hotel Name
22,bethel,US,41.3712,-73.4140,Hampton Inn Danbury
90,corfu,GR,39.6200,19.9197,Hotel Bella Venezia
101,tamanrasset,DZ,22.7850,5.5228,فندق الأمان
129,colonia,DE,50.9333,6.9500,Wasserturm Hotel Cologne
140,farah,AF,32.5000,62.5000,No hotel found
215,brookings,US,44.3114,-96.7984,Quality Inn
232,revelstoke,CA,50.9832,-118.2023,Grizz Hotel
238,saint-pierre,RE,-21.3393,55.4781,Tropic Hotel
253,sedalia,US,38.7045,-93.2283,Hotel Bothwell
285,santa terezinha,BR,-29.9002,-50.0904,Pousada Santa Terezinha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel", "Country"]
    
)

map_plot

# Display the map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country)